In [47]:
import plotly.plotly as py
import plotly.graph_objs as go
import csv, numpy
from datetime import datetime

In [48]:
class GNP_DataPoint:
    def __init__(self, csv_dict):
        """Converts csv_dict to an instance of GNP_DataPoint."""
        if 'DATE' in csv_dict.keys():
            conven_date = [int(string) for string in csv_dict['DATE'].split('/')]
            conven_date[2] = conven_date[2] + 1900 if conven_date[2] > 17 else conven_date[2] + 2000
            csv_dict['DATE'] = datetime(year = conven_date[2], month = conven_date[0], day = conven_date[1])
        self.data = csv_dict
    
    def get_data(self, req):
        """Given a list of requests, returns requested data from self.data."""
        re = []
        for query in req:
            try:
                if not self.data[query] == 'None':
                    re.append(self.data[query])
            except KeyError:
                pass
        return re
    
    def __repr__(self):
        return str(self.data)
    
class GNP_Database:
    def __init__(self):
        """Instantiates GNP_Database."""
        self.database = []
    
    def add_point(self, csv_dict):
        """Adds GNP_DataPoint based on csv_dict to self.database"""
        self.database.append(GNP_DataPoint(csv_dict))
    
    def request(self, req):
        """Returns data requested in list req"""
        re = []
        for point in self.database:
            re.extend(point.get_data(req))
        return re
    
    def __iter__(self):
        for point in self.database:
            yield point
    
    def __getitem__(self, key):
        return self.request([key])

In [72]:
gnp_data = GNP_Database()
with open('data/freddata.csv', 'r') as csvfile: 
    gnpdata = csv.DictReader(csvfile)
    for row in gnpdata:
        gnp_data.add_point(row)

In [43]:
x = gnp_data['DATE']

trace1 = go.Scatter(
    x = x,
    y = gnp_data['GNP'],
    name = 'GNP'
)

trace2 = go.Scatter(
    x = x,
    y = gnp_data['CB-GNP'],
    name = 'Command Basis GNP'
)

data = [trace1, trace2]
py.iplot(data)

In [44]:
trace3 = go.Scatter(
    x = x,
    y = gnp_data['GNPYC'],
    name = 'GNP (% Change from Last Year)'
)

trace4 = go.Scatter(
    x = x,
    y = gnp_data['CB-GNPYC'],
    name = 'Command Basis GNP (% Change from Last Year)'
)

data = [trace3, trace4]
py.iplot(data)

In [54]:
adjusted_date_1988 = [date for date in x if date.year >= 1988]
trace5 = go.Scatter(
    x = adjusted_date_1988,
    y = gnp_data['OUTMS'],
    name = 'Manufacturing: Real Ouput (Index 2009 = 100)'
)

trace6 = go.Scatter(
    x = adjusted_date_1988,
    y = gnp_data['OUTNFB'],
    name = 'Nonfarm Business: Real Ouput (Index 2009 = 100)'
)

data = [trace5, trace6]
py.iplot(data)

In [46]:
trace7 = go.Scatter(
    x = adjusted_date_1988,
    y = gnp_data['OPHMFG'],
    name = 'Manufacturing: Real Ouput Per Hour Per Worker (Index 2009 = 100)'
)

trace8 = go.Scatter(
    x = adjusted_date_1988,
    y = gnp_data['OPHNFB'],
    name = 'Nonfarm Business: Real Ouput Per Hour Per Worker(Index 2009 = 100)'
)

data = [trace7, trace8]
py.iplot(data)

In [75]:
adjusted_date_1960_2015 = [date for date in x if date.year >= 1960 and date.year <= 2015 and date.month == 1]
adjusted_date_1950_2011 = [date for date in x if date.year >= 1950 and date.year <= 2011 and date.month == 1]
trace9 = go.Scatter(
    x = adjusted_date_1960_2015,
    y = gnp_data['T-GDP'],
    name = 'Trade (% of GDP)'
)
trace10 = go.Scatter(
    x = adjusted_date_1950_2011,
    y = gnp_data['OUTMS-A'],
    name = 'Manufacturing Output',
    yaxis = 'y2'
)
data = [trace9, trace10]

layout = go.Layout(
    title='Trade vs Manufactuing Ouput',
    showlegend=True,
    legend=dict(
        x=0,
        y=1
    ),
    yaxis=dict(
        title='Percent of GDP'
    ),
    yaxis2=dict(
        title='Index (2009 = 100)', 
        overlaying='y',
        side='right'
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [83]:
trace10 = go.Scatter(
    x = x,
    y = gnp_data['MANEMP'],
    name = 'Employment'
)
trace11 = go.Scatter(
    x = adjusted_date_1950_2011,
    y = gnp_data['OUTMS-A'],
    name = 'Real Output',
    yaxis = 'y2'
)
trace7_revised = go.Scatter(
    x = adjusted_date_1988,
    y = gnp_data['OPHMFG'],
    name = 'Ouput Per Worker',
    yaxis = 'y2'
)
data = [trace10, trace11, trace7_revised]

layout = go.Layout(
    title='Manufacturing Trends',
    showlegend=True,
    legend=dict(
        orientation = 'h'
    ),
    yaxis=dict(
        title='Thousands of Persons'
    ),
    yaxis2=dict(
        title='Index (2009 = 100)', 
        overlaying='y',
        side='right'
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)